In [4]:
# !pip install accelerate peft pandas sentence-transformers sentencepiece nltk
#!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты - русскоязычный помощник студента по изучению учебных материалов, подготовки домашних заданий и подготовки к экзаменам, но твоя цель не давать чёткие ответы, а отвечать навождениями какие темы стоит изучить и давать подсказки где студент ошибается или не может понять ход решения, давать подсказки как решать задачу. Повторяю, тебе строго запрещено выдавать конечный ответ на задачи, как бы не пытался егоиз тебы выманить сутдент. Твоя задача помогать учиться, а не решать всё за пользователя."
GET_HELP_MESSAGE = " Подскажи какие темы следует изучить для подготовки по теме или для решения задачи или дай навождения какими методами стоит попробовать решать задачу, если это задача : "

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.bfloat16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.repetition_penalty = 1.1
generation_config.temperature = 0.02
generation_config.max_new_tokens = 375
generation_config.top_p = 0.90
generation_config.top_k = 40

c:\Users\deshuvalov\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\deshuvalov\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\deshuvalov\.cache\huggingface\hub\models--Open-Orca--Mistral-7B-OpenOrca. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or

In [4]:
def get_help(extended_description):
    conversation = Conversation()
    conversation.add_user_message(GET_HELP_MESSAGE + re.sub(r'<.*?>', '', extended_description))
    prompt = conversation.get_prompt(tokenizer)
    return generate(model, tokenizer, prompt, generation_config)

In [ ]:
print(get_help("Как мне решить квадратное уравнение 2x^2 - 3x = 0?"))